SQL Project

-----------------------------------------------------
**Project goal**

Explore and analyze this database that contains data on books, publishers, authors, customer ratings and reviews of books.

This information will be used to generate a value proposition for a new products in the furue.

--------------------------------------

**Description**

    - The projects explores the database if the followinf steps:
    
    
- Find the number of books released after January 1, 2000.


- Find the number of user reviews and the average rating for each book.


- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude  brochures and similar publications from your analysis).


- Identify the author with the highest average book rating: look only at books with at least 50 ratings.


- Find the average number of text reviews among users who rated more than 50 books.

In [1]:
pip install psycopg2-binary 

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

 --- how to write a qury on jupyter :
 
pd.io.sql.read_sql(  #### , con = engine)

**Description of the data**

- books:

Contains data on books:

book_id

author_id

title

num_pages — number of pages


publication_date

publisher_id

- authors:

Contains data on authors:

author_id

author

- publishers:

Contains data on publishers:

publisher_id

publisher

- ratings:

Contains data on user ratings:

rating_id

book_id

username — the name of the user who rated the book

rating

- reviews:

Contains data on customer reviews:

review_id

book_id

username — the name of the user who reviewed the book

text — the text of the review

----------------------------------------------

In [ ]:
# - Find the number of books released after January 1, 2000.

sql_query ="""

SELECT *
FROM books
WHERE CAST(publication_date AS date) >= '2000-1-1'

"""
num_book = pd.io.sql.read_sql(sql_query , con = engine)
num_book

Now we know that there are 821 books that got released since January 1, 2000.

-----------------------------

In [ ]:
# - Find the number of user reviews and the average rating for each book.

sql_query ="""

SELECT title,
       COUNT(rating_id) AS n_reviews,
       AVG(rating) AS avg_rating
       
FROM ratings AS r
JOIN books AS b ON r.book_id=b.book_id
GROUP BY r.book_id,title
ORDER BY avg_rating DESC;
"""
num_user_reviews = pd.io.sql.read_sql(sql_query , con = engine)
num_user_reviews

---------------------

In [ ]:
#  - Identify the publisher that has released the greatest number of books with more than 50 pages.
# (this will help you exclude brochures and similar publications from your analysis).

sql_query="""

SELECT 
       COUNT(book_id) AS books,
       publisher
FROM publishers AS p
JOIN books AS b ON p.publisher_id=b.publisher_id
WHERE num_pages >50
GROUP BY publisher
ORDER BY books DESC;
"""
lead_publisher = pd.io.sql.read_sql(sql_query , con = engine)
lead_publisher

Of course it would be penguin books, with 42 books.

In [ ]:
# - Identify the author with the highest average book rating: look only at books with at least 50 ratings.
sql_query= """

SELECT author,AVG(rate_of)
FROM (SELECT b.book_id,b.author_id,AVG(r.rating) AS rate_of
    FROM books AS b
LEFT JOIN ratings AS r ON r.book_id=b.book_id
    GROUP BY b.book_id
    HAVING COUNT(r.rating)>50
    ORDER BY AVG(r.rating)) AS f
LEFT JOIN authors AS a ON f.author_id = a.author_id
GROUP BY author
LIMIT 1;
"""
lead_author = pd.io.sql.read_sql(sql_query , con = engine)
lead_author

In [ ]:
# - Find the average number of text reviews among users who rated more than 50 books.
sql_query="""

SELECT AVG(no_reviews) AS avg_num
    
FROM( 
    SELECT COUNT(r.review_id) AS no_reviews      
      FROM
      (SELECT username
      FROM ratings
      GROUP BY username
      HAVING COUNT(rating_id)>50) AS a
JOIN reviews AS r ON a.username = r.username
GROUP BY a.username) AS f;


"""
lead_user = pd.io.sql.read_sql(sql_query , con = engine)
lead_user

 the average number of text reviews among users who rated more than 50 books is 24.3

**Summary**

- there are 821 books that got released since January 1, 2000.

- Theres 1000 booksin the data 

- Pinguin house is the publisher that has released the greatest number of books with more than 50 pages. 

- 

-  the average number of text reviews among users who rated more than 50 books is 24.3